# Dados Principais

In [ ]:
import pandas as pd
import threading 
 
df1 = pd.read_csv('/content/drive/MyDrive/ic/Dados - Indivíduos SNPs/adni_gwas_v2/adni_gwas_v2_set01/002_S_0295.csv')
#df2= pd.read_csv('/content/drive/MyDrive/ic/Dados - Indivíduos SNPs/adni_gwas_v2/adni_gwas_v2_set01/002_S_0413.csv')

df1 =df1[df1['SNP Name'].str.startswith('rs')]

#tirar_duplicados = diag.drop_duplicates('Subject',keep='first')
df1[['SNP Name', 'Allele1 - AB','Allele2 - AB']]
#esse print contêm todos os SNPS encontrados em um individuo 

,SNP Name,Allele1 - AB,Allele2 - AB
22080,rs1000000,B,B
22081,rs10000010,A,B
22082,rs10000023,A,B
22083,rs10000030,B,B
22084,rs10000041,A,A
...,...,...,...
620896,rs9999944,A,B
620897,rs999995,B,B
620898,rs9999963,A,A
620899,rs9999966,B,B


In [ ]:
import os #sistema operacioal 
folders = os.listdir('/content/drive/My Drive/ic/Dados - Indivíduos SNPs/adni_gwas_v2')
folders.sort()
folders

['adni_gwas_v2_set01',
 'adni_gwas_v2_set02',
 'adni_gwas_v2_set03',
 'adni_gwas_v2_set04',
 'adni_gwas_v2_set05',
 'adni_gwas_v2_set06',
 'adni_gwas_v2_set07',
 'adni_gwas_v2_set08',
 'adni_gwas_v2_set09',
 'adni_gwas_v2_set10',
 'adni_gwas_v2_set11']

# Funções que vão ser usadas em Thread




In [ ]:
##Filtrando apenas os dados q começam com rs 
def Filtragem(arg):
  arg =arg[arg['SNP Name'].str.startswith('rs')]
  return arg

In [ ]:
xlsx=pd.read_csv('/content/drive/My Drive/ic/NomesSnpsEAlelos.csv') #leitura dos os snps e seus respectivos genes
xlsx = xlsx.transpose()
xlsx

,0,1
Unnamed: 0,Name,Alleles
0,rs7412,SNV:C>T\n
1,rs405509,SNV:T>G\n
2,rs429358,SNV:T>C\n
3,rs769452,SNV:T>C\n
...,...,...
147312,rs796829949,"DELINS:ATTTA>-,ATTTAATTTA\n"
147313,rs1416914868,"DELINS:AATT>-,AATTAATT,AATTAATTAATT,AATTAATTAA..."
147314,rs1400255377,"DELINS:TT>-,TTT\n"
147315,rs1491287923,"DELINS:AT>-,ATAT,ATATAT,ATATATAT,ATATATATATAT,..."


In [ ]:
csv = pd.read_csv(f'/content/drive/My Drive/ic/Dados - Indivíduos SNPs/adni_gwas_v2/adni_gwas_v2_set01/002_S_0295.csv',usecols=["SNP Name","Allele1 - AB","Allele2 - AB"]) 
csv = Filtragem(csv)
#print(csv)

meu_indexador = csv["SNP Name"].isin(xlsx[0])
       
resultado_sem_comparar = csv[meu_indexador].transpose()
#resultado_sem_comparar.columns = range(resultado_sem_comparar.shape[1])
print( resultado_sem_comparar)

                  30399       34538   ...     594960    614271
SNP Name      rs10127904  rs10194375  ...  rs9473121  rs988337
Allele1 - AB           A           B  ...          A         A
Allele2 - AB           B           B  ...          B         A

[3 rows x 126 columns]


In [ ]:
def LeituraArquivos(folders):
  result = pd.DataFrame()
  for folder in folders:
    #file = list()
    file = os.listdir(f'/content/drive/My Drive/ic/Dados Descompactados adni/adni_gwas_v2/{folder}') #procura mais profundamente todos os .csv em todas as pastas indicadas
    file.sort()
    for id in file: #pego cada um dos cerca de 200 arquivos
        csv = pd.read_csv(f'/content/drive/My Drive/ic/Dados Descompactados adni//adni_gwas_v2/{folder}/{id}',usecols=["SNP Name","Allele1 - AB","Allele2 - AB"]) 
        csv = Filtragem(csv)
        #print(csv)

        meu_indexador = csv["SNP Name"].isin(xlsx[0])
       
        resultado_sem_comparar = csv[meu_indexador].transpose()
        resultado_sem_comparar.columns = range(resultado_sem_comparar.shape[1])
        print( resultado_sem_comparar)

        result = pd.concat([result,compare(resultado_sem_comparar,id)])
        #print(result)´
  print('acabou')
  return pd.get_dummies(result, prefix=result.columns) #one-hot-encoding

In [ ]:
#Compara allele1-AB com allele2-AB
import numpy as np

def compare(dataframe,individuo):
  result = pd.Series(np.full(dataframe.columns.size,3),dtype='int64',name=f'{individuo}'.replace('.csv',''))

  result[(dataframe.iloc[1,:]==dataframe.iloc[2,:]) & (dataframe.iloc[1,:]=='A')] = 1
  result[(dataframe.iloc[1,:]==dataframe.iloc[2,:]) & (dataframe.iloc[1,:]=='B')] = 2
  result = result.to_frame()
  

  result = pd.concat([dataframe,result.transpose()]).drop(['Allele1 - AB','Allele2 - AB'])
  result.columns = result.iloc[0]
  result= result.drop('SNP Name')
  
  
  return result 

In [ ]:
LeituraArquivos(folders[0:1])

In [ ]:
import concurrent.futures 
with concurrent.futures.ThreadPoolExecutor() as executor:
    future = executor.submit(LeituraArquivos,folders[0:(int(len(folders)/2))])
    return_value = future.result()

with concurrent.futures.ThreadPoolExecutor() as executor:
    future2 = executor.submit(LeituraArquivos,folders[int(len(folders)/2):len(folders)])
    return_value2 = future2.result()

cabou
cabou


In [ ]:
print(return_value)

            rs10127904_1  rs10127904_2  ...  rs9473121_3  rs988337_1
002_S_0295             0             0  ...            1           1
002_S_0413             1             0  ...            1           1
002_S_0559             0             0  ...            0           1
002_S_0619             1             0  ...            0           1
002_S_0685             1             0  ...            1           1
...                  ...           ...  ...          ...         ...
037_S_0588             0             0  ...            1           1
037_S_0627             0             0  ...            0           1
037_S_1078             1             0  ...            0           1
037_S_1225             0             0  ...            1           1
037_S_1421             1             0  ...            0           1

[368 rows x 353 columns]


In [ ]:
print(return_value2)

            rs10127904_1  rs10127904_2  ...  rs9473121_3  rs988337_1
041_S_0125             1             0  ...            0           1
041_S_0262             0             0  ...            1           1
041_S_0282             1             0  ...            0           1
041_S_0314             1             0  ...            0           1
041_S_0407             0             0  ...            0           1
...                  ...           ...  ...          ...         ...
941_S_1202             0             0  ...            0           1
941_S_1203             1             0  ...            1           1
941_S_1295             0             0  ...            0           1
941_S_1311             0             0  ...            0           1
941_S_1363             1             0  ...            1           1

[389 rows x 354 columns]


In [ ]:
final_to_csv = pd.concat([return_value,return_value2])

In [ ]:
final_to_csv.to_csv('/content/drive/My Drive/ic/extracao_atributes.csv')

#Repetição com novos dados direto do banco de dados

In [ ]:
snpstop10=pd.read_csv('/content/drive/MyDrive/ic/Dados limpos/snps_top10_onecolumn.csv',usecols=['snp']) #leitura dos os snps e seus respectivos genes
snpstop10['snp'] = snpstop10['snp'].astype(str)
snpstop10

,snp
0,1573856032
1,1573856015
2,1573855660
3,1573855588
4,1573855529
...,...
154463,435380
154464,434132
154465,429358
154466,405509


In [ ]:
##Filtrando apenas os dados q começam com rs 
def Filtragemnew(arg):
  arg =arg[arg['SNP Name'].str.startswith('rs')]
  arg['SNP Name'] = arg['SNP Name'].str.replace('rs', '')
  return arg

In [ ]:
def LeituraArquivosnew(folders):
  result = pd.DataFrame()
  for folder in folders:

    file = os.listdir(f'/content/drive/My Drive/ic/Dados - Indivíduos SNPs/adni_gwas_v2/{folder}') #procura mais profundamente todos os .csv em todas as pastas indicadas
    file.sort()
    for id in file: #pego cada um dos cerca de 200 arquivos
        csv = pd.read_csv(f'/content/drive/My Drive/ic/Dados - Indivíduos SNPs/adni_gwas_v2/{folder}/{id}',usecols=["SNP Name","Allele1 - AB","Allele2 - AB"]) 
        csv = Filtragemnew(csv)

        meu_indexador = csv["SNP Name"].isin(snpstop10['snp'])
       
        resultado_sem_comparar = csv[meu_indexador].transpose()
        resultado_sem_comparar.columns = range(resultado_sem_comparar.shape[1])
        print(resultado_sem_comparar)

        result = pd.concat([result,comparenew(resultado_sem_comparar,id)])
        print('O Resultado de cada individuo com o hot enocding: \n')
        print(result)
  print('acabou')
  return pd.get_dummies(result, prefix=result.columns) #one-hot-encoding

In [ ]:
#Compara allele1-AB com allele2-AB
import numpy as np

def comparenew(dataframe,individuo):
  result = pd.Series(np.full(dataframe.columns.size,3),dtype='int64',name=f'{individuo}'.replace('.csv',''))

  result[(dataframe.iloc[1,:]==dataframe.iloc[2,:]) & (dataframe.iloc[1,:]=='A')] = 1
  result[(dataframe.iloc[1,:]==dataframe.iloc[2,:]) & (dataframe.iloc[1,:]=='B')] = 2
  result = result.to_frame()
  

  result = pd.concat([dataframe,result.transpose()]).drop(['Allele1 - AB','Allele2 - AB'])
  result.columns = result.iloc[0]
  result= result.drop('SNP Name')
  
  
  return result 

In [ ]:
import concurrent.futures 
with concurrent.futures.ThreadPoolExecutor() as executor:
    future = executor.submit(LeituraArquivosnew,folders[0:(int(len(folders)/2))])
    return_value_new = future.result()

with concurrent.futures.ThreadPoolExecutor() as executor:
    future2 = executor.submit(LeituraArquivosnew,folders[int(len(folders)/2):len(folders)])
    return_value2_new = future2.result()

Os dados são:
*     AA -> 1 -> homozigoto com mutação 
*     BB -> 2 -> homizigoto com mutação 
*     AB -> 3 -> heterozigo 


In [ ]:
parasalvar = LeituraArquivosnew(folders)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


                   0         1         2    ...      123      124     125
SNP Name      10127904  10194375  10200967  ...  9395285  9473121  988337
Allele1 - AB         A         B         A  ...        B        A       A
Allele2 - AB         B         B         B  ...        B        B       A

[3 rows x 126 columns]
O Resultado de cada individuo com o hot enocding: 

SNP Name   10127904 10194375 10200967 10419707  ... 9395267 9395285 9473121 988337
002_S_0295        3        2        3        1  ...       2       2       3      1

[1 rows x 126 columns]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


                   0         1         2    ...      123      124     125
SNP Name      10127904  10194375  10200967  ...  9395285  9473121  988337
Allele1 - AB         A         B         B  ...        B        A       A
Allele2 - AB         A         B         B  ...        B        B       A

[3 rows x 126 columns]
O Resultado de cada individuo com o hot enocding: 

SNP Name   10127904 10194375 10200967 10419707  ... 9395267 9395285 9473121 988337
002_S_0295        3        2        3        1  ...       2       2       3      1
002_S_0413        1        2        2        1  ...       3       2       3      1

[2 rows x 126 columns]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


...

                   0         1         2    ...      123      124     125
SNP Name      10127904  10194375  10200967  ...  9395285  9473121  988337
Allele1 - AB         A         B         A  ...        B        A       A
Allele2 - AB         B         B         B  ...        B        A       A

[3 rows x 126 columns]
O Resultado de cada individuo com o hot enocding: 

SNP Name   10127904 10194375 10200967 10419707  ... 9395267 9395285 9473121 988337
002_S_0295        3        2        3        1  ...       2       2       3      1
002_S_0413        1        2        2        1  ...       3       2       3      1
002_S_0559        3        2        1        1  ...       2       3       2      1
002_S_0619        1        2        2        1  ...       2       2       2      1
002_S_0685        1        3        1        1  ...       2       2       3      1
...             ...      ...      ...      ...  ...     ...     ...     ...    ...
941_S_1195        3        2        3        1 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


                   0         1         2    ...      123      124     125
SNP Name      10127904  10194375  10200967  ...  9395285  9473121  988337
Allele1 - AB         A         B         A  ...        A        B       A
Allele2 - AB         B         B         B  ...        A        B       A

[3 rows x 126 columns]
O Resultado de cada individuo com o hot enocding: 

SNP Name   10127904 10194375 10200967 10419707  ... 9395267 9395285 9473121 988337
002_S_0295        3        2        3        1  ...       2       2       3      1
002_S_0413        1        2        2        1  ...       3       2       3      1
002_S_0559        3        2        1        1  ...       2       3       2      1
002_S_0619        1        2        2        1  ...       2       2       2      1
002_S_0685        1        3        1        1  ...       2       2       3      1
...             ...      ...      ...      ...  ...     ...     ...     ...    ...
941_S_1197        1        3        1        1 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


                   0         1         2    ...      123      124     125
SNP Name      10127904  10194375  10200967  ...  9395285  9473121  988337
Allele1 - AB         A         A         A  ...        A        A       A
Allele2 - AB         A         A         A  ...        B        B       A

[3 rows x 126 columns]
O Resultado de cada individuo com o hot enocding: 

SNP Name   10127904 10194375 10200967 10419707  ... 9395267 9395285 9473121 988337
002_S_0295        3        2        3        1  ...       2       2       3      1
002_S_0413        1        2        2        1  ...       3       2       3      1
002_S_0559        3        2        1        1  ...       2       3       2      1
002_S_0619        1        2        2        1  ...       2       2       2      1
002_S_0685        1        3        1        1  ...       2       2       3      1
...             ...      ...      ...      ...  ...     ...     ...     ...    ...
941_S_1202        3        1        1        1 

In [ ]:
csv = pd.read_csv(f'/content/drive/My Drive/ic/Dados - Indivíduos SNPs/adni_gwas_v2/adni_gwas_v2_set01/002_S_0295.csv',usecols=["SNP Name","Allele1 - AB","Allele2 - AB"]) 
csv = Filtragemnew(csv)
#print(csv)
csv['SNP Name'] = csv['SNP Name'].str.replace('rs', '')
#print(csv[csv['SNP Name']== '405509'])

meu_indexador = csv["SNP Name"].isin(snpstop10['snp'])

resultado_sem_comparar = csv[meu_indexador]
#resultado_sem_comparar.columns = range(resultado_sem_comparar.shape[1])
print( resultado_sem_comparar)